<a href="https://colab.research.google.com/github/khoubjimahmoud/attention_tracker0/blob/main/attention_tracker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imports

In [8]:
import os
from google.colab import files
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [10]:
uploaded = files.upload()

# Assuming the file name is 'annotexample.csv'
file_path = next(iter(uploaded.keys()))
annotations = pd.read_csv(file_path)
annotations.head()

Saving _annotations.csv to _annotations.csv


,filename,width,height,class,xmin,ymin,xmax,ymax
0,img1817_jpg.rf.eda65b21c04df3528d3df8ff8eac425...,220,220,Normal,50,28,162,177
1,user_03_video_06_img55_jpg.rf.e96af72b76ee7e61...,1280,720,Normal,529,203,794,558
2,img1731_jpg.rf.ed6f1f77bed21ae24a207db16db1a22...,220,220,Normal,46,28,161,165
3,img1890_jpg.rf.eb863163ab7fa8ba164a7a2e14e789c...,220,220,Normal,46,27,162,178
4,user_01_video_06_img228_jpg.rf.eaafc883a2b66f7...,1280,720,LookDown,578,181,828,523


In [11]:
# Import necessary libraries
import os
import numpy as np
import pandas as pd  # Assuming you need pandas for DataFrame operations
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

from google.colab import drive
drive.mount('/content/drive')

# Set the image dimensions and path
img_height, img_width = 128, 128
image_dir = '/content/drive/MyDrive/My Drive/samples/'  # Path to the extracted images

# Create lists to store the images and labels
images = []
labels = []

# Assuming you have a DataFrame named `annotations` with columns 'filename' and 'class'
# Load your annotations DataFrame (replace this with your actual DataFrame loading code)
annotations = pd.read_csv('/content/_annotations.csv')

# Define the binary label conversion function
def convert_label(label):
    return 0 if label == "Normal" else 1

# Load and preprocess the images
for idx, row in annotations.iterrows():
    img_path = os.path.join(image_dir, row['filename'])
    if os.path.exists(img_path):
        img = load_img(img_path, target_size=(img_height, img_width))
        img_array = img_to_array(img)
        images.append(img_array)
        labels.append(convert_label(row['class']))
    else:
        print(f"Image {img_path} not found.")

# Convert lists to numpy arrays
images = np.array(images)
labels = np.array(labels)

# Check the number of images and labels
print(f"Number of images: {len(images)}")
print(f"Number of labels: {len(labels)}")

# Ensure there are enough samples
if len(images) == 0 or len(labels) == 0:
    raise ValueError("No images or labels found. Please check the annotations and image paths.")

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

# Normalize the pixel values to be between 0 and 1
X_train = X_train / 255.0
X_val = X_val / 255.0

# Build the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()

# Train the model
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val), callbacks=[early_stopping])

# Save the model
model.save('/content/model.h5')
model.save('/content/drive/MyDrive/My Drive/model.h5')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Number of images: 1111
Number of labels: 1111
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 63, 63, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 30, 30, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Co

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from google.colab import files
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

In [ ]:
uploaded = files.upload()
sample_image_path = next(iter(uploaded.keys()))

Saving A.png to A.png


In [ ]:
model = load_model('/content/model.h5')

# Step 3: Preprocess the Sample Image
img_height, img_width = 128, 128
sample_img = load_img(sample_image_path, target_size=(img_height, img_width))
sample_img_array = img_to_array(sample_img)
sample_img_array = sample_img_array / 255.0  # Normalize pixel values
sample_img_array = np.expand_dims(sample_img_array, axis=0)  # Add batch dimension

# Step 4: Make Predictions
predictions = model.predict(sample_img_array)
prediction_class = 'Normal' if predictions[0][0] < 0.5 else 'Other'

print(f'Prediction: {prediction_class}')

1/1 [==============================] - 0s 118ms/step
Prediction: Other


Second Version

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import cv2
import numpy as np
from keras.models import load_model
from keras.preprocessing.image import img_to_array, load_img

# Define a function to extract frames from the video
def extract_frames(video_path, interval=30):
    frames = []
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print(f"Error: Cannot open video file {video_path}")
        return frames

    frame_rate = cap.get(cv2.CAP_PROP_FPS)
    if frame_rate == 0:
        print("Error: Frame rate is zero.")
        cap.release()
        return frames

    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    for i in range(0, frame_count, int(frame_rate * interval)):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i)
        ret, frame = cap.read()
        if ret:
            frames.append(frame)
        else:
            break

    cap.release()
    return frames

# Define a function to preprocess the frame
def preprocess_frame(frame, img_height=128, img_width=128):
    frame = cv2.resize(frame, (img_height, img_width))
    frame = frame / 255.0  # Normalize pixel values
    frame = np.expand_dims(frame, axis=0)  # Add batch dimension
    return frame

# Define a function to classify frames and calculate the percentage of "Normal" frames
def classify_frames(frames, model):
    normal_count = 0
    for frame in frames:
        processed_frame = preprocess_frame(frame)
        predictions = model.predict(processed_frame)
        if predictions[0][0] < 0.5:
            normal_count += 1

    if len(frames) == 0:
        return 0

    normal_percentage = (normal_count / len(frames)) * 100
    return normal_percentage

# Load the pre-trained model
model_path = '/content/model.h5'  # Adjust the path to the model file as needed
model = load_model(model_path)

# Path to the video file on Google Drive
video_path = '/content/drive/My Drive/My Drive/sample_vid.mp4'

# Extract frames from the video
frames = extract_frames(video_path)

# Classify the frames and calculate the percentage of "Normal" frames
if frames:
    normal_percentage = classify_frames(frames, model)
    print(f"Percentage of 'Normal' frames: {normal_percentage}%")
else:
    print("No frames extracted from the video.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
1/1 [==============================] - 0s 31ms/step
Percentage of 'Normal' frames: 33.33333333333333%


Last Version (wa9ila)

Extract frames

In [12]:
import cv2

def extract_frames(video_path, interval=30):
    frames = []
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print(f"Error: Cannot open video file {video_path}")
        return frames

    frame_rate = cap.get(cv2.CAP_PROP_FPS)
    if frame_rate == 0 or frame_rate is None:
        print("Error: Frame rate is zero or could not be determined.")
        cap.release()
        return frames

    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    for i in range(0, frame_count, int(frame_rate * interval)):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i)
        ret, frame = cap.read()
        if ret:
            frames.append(frame)
        else:
            break

Detect faces

In [13]:
def detect_faces(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)
    return faces


Track Faces

In [14]:
from collections import OrderedDict
import numpy as np

class CentroidTracker:
    def __init__(self, maxDisappeared=50):
        self.nextObjectID = 0
        self.objects = OrderedDict()
        self.disappeared = OrderedDict()
        self.maxDisappeared = maxDisappeared

    def register(self, centroid):
        self.objects[self.nextObjectID] = centroid
        self.disappeared[self.nextObjectID] = 0
        self.nextObjectID += 1

    def deregister(self, objectID):
        del self.objects[objectID]
        del self.disappeared[objectID]

    def update(self, rects):
        if len(rects) == 0:
            for objectID in list(self.disappeared.keys()):
                self.disappeared[objectID] += 1
                if self.disappeared[objectID] > self.maxDisappeared:
                    self.deregister(objectID)
            return self.objects

        inputCentroids = np.zeros((len(rects), 2), dtype="int")

        for (i, (startX, startY, endX, endY)) in enumerate(rects):
            cX = int((startX + endX) / 2.0)
            cY = int((startY + endY) / 2.0)
            inputCentroids[i] = (cX, cY)

        if len(self.objects) == 0:
            for i in range(0, len(inputCentroids)):
                self.register(inputCentroids[i])
        else:
            objectIDs = list(self.objects.keys())
            objectCentroids = list(self.objects.values())

            D = dist.cdist(np.array(objectCentroids), inputCentroids)
            rows = D.min(axis=1).argsort()
            cols = D.argmin(axis=1)[rows]

            usedRows = set()
            usedCols = set()

            for (row, col) in zip(rows, cols):
                if row in usedRows or col in usedCols:
                    continue

                objectID = objectIDs[row]
                self.objects[objectID] = inputCentroids[col]
                self.disappeared[objectID] = 0

                usedRows.add(row)
                usedCols.add(col)

            unusedRows = set(range(0, D.shape[0])).difference(usedRows)
            unusedCols = set(range(0, D.shape[1])).difference(usedCols)

            if D.shape[0] >= D.shape[1]:
                for row in unusedRows:
                    objectID = objectIDs[row]
                    self.disappeared[objectID] += 1

                    if self.disappeared[objectID] > self.maxDisappeared:
                        self.deregister(objectID)
            else:
                for col in unusedCols:
                    self.register(inputCentroids[col])

        return self.objects


Attention analysis

In [15]:
from keras.models import load_model

# Load the pre-trained model
model_path = '/content/model.h5'  # Adjust the path to the model file as needed
model = load_model(model_path)

def preprocess_frame(frame, img_height=128, img_width=128):
    frame = cv2.resize(frame, (img_height, img_width))
    frame = frame / 255.0  # Normalize pixel values
    frame = np.expand_dims(frame, axis=0)  # Add batch dimension
    return frame

def classify_frame(model, frame):
    processed_frame = preprocess_frame(frame)
    predictions = model.predict(processed_frame)
    return predictions[0][0] < 0.5  # True if classified as "Normal"


Final percentage generation

In [16]:
!pip install pytesseract
!apt-get install -y tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 5s (1,016 kB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 121925 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-

In [17]:
import cv2
import numpy as np
from keras.models import load_model
from google.colab import drive
from scipy.spatial import distance as dist
from google.colab.patches import cv2_imshow
import time

# Mount Google Drive
drive.mount('/content/drive')

# Initialize the Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Pre-trained model path
model_path = '/content/model.h5'  # Adjust the path to the model file as needed
model = load_model(model_path)

# Define the CentroidTracker class
class CentroidTracker:
    def __init__(self, maxDisappeared=50):
        self.nextObjectID = 0
        self.objects = {}
        self.disappeared = {}
        self.maxDisappeared = maxDisappeared

    def register(self, centroid):
        self.objects[self.nextObjectID] = centroid
        self.disappeared[self.nextObjectID] = 0
        self.nextObjectID += 1

    def deregister(self, objectID):
        del self.objects[objectID]
        del self.disappeared[objectID]

    def update(self, rects):
        if len(rects) == 0:
            for objectID in list(self.disappeared.keys()):
                self.disappeared[objectID] += 1
                if self.disappeared[objectID] > self.maxDisappeared:
                    self.deregister(objectID)
            return self.objects

        inputCentroids = np.zeros((len(rects), 2), dtype="int")
        for (i, (startX, startY, endX, endY)) in enumerate(rects):
            cX = int((startX + endX) / 2.0)
            cY = int((startY + endY) / 2.0)
            inputCentroids[i] = (cX, cY)

        if len(self.objects) == 0:
            for i in range(0, len(inputCentroids)):
                self.register(inputCentroids[i])
        else:
            objectIDs = list(self.objects.keys())
            objectCentroids = list(self.objects.values())

            D = dist.cdist(np.array(objectCentroids), inputCentroids)
            rows = D.min(axis=1).argsort()
            cols = D.argmin(axis=1)[rows]

            usedRows = set()
            usedCols = set()

            for (row, col) in zip(rows, cols):
                if row in usedRows or col in usedCols:
                    continue

                objectID = objectIDs[row]
                self.objects[objectID] = inputCentroids[col]
                self.disappeared[objectID] = 0

                usedRows.add(row)
                usedCols.add(col)

            unusedRows = set(range(0, D.shape[0])).difference(usedRows)
            unusedCols = set(range(0, D.shape[1])).difference(usedCols)

            if D.shape[0] >= D.shape[1]:
                for row in unusedRows:
                    objectID = objectIDs[row]
                    self.disappeared[objectID] += 1

                    if self.disappeared[objectID] > self.maxDisappeared:
                        self.deregister(objectID)
            else:
                for col in unusedCols:
                    self.register(inputCentroids[col])

        return self.objects

# Define the detect_faces function
def detect_faces(frame):
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    return faces

# Define the preprocess_frame function
def preprocess_frame(frame, img_height=128, img_width=128):
    frame = cv2.resize(frame, (img_height, img_width))
    frame = frame / 255.0  # Normalize pixel values
    frame = np.expand_dims(frame, axis=0)  # Add batch dimension
    return frame

# Define the classify_frame function
def classify_frame(model, face_img):
    processed_frame = preprocess_frame(face_img)
    predictions = model.predict(processed_frame)
    return predictions[0][0] < 0.5

# Define a function to calculate attention percentage for each user in 2-minute clips
def calculate_attention_percentage(video_path, clip_length=120, pause_length=120, frame_interval=10):
    cap = cv2.VideoCapture(video_path)
    frame_rate = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    clip_frames = int(frame_rate * clip_length)
    pause_frames = int(frame_rate * pause_length)
    frame_step = int(frame_rate * frame_interval)
    tracker = CentroidTracker()
    attention_counts = {}
    total_counts = {}

    frame_number = 0
    while frame_number < total_frames:
        if frame_number % (clip_frames + pause_frames) < clip_frames:
            cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
            ret, frame = cap.read()
            if not ret:
                break

            faces = detect_faces(frame)
            rects = [(x, y, x+w, y+h) for (x, y, w, h) in faces]
            objects = tracker.update(rects)

            for (objectID, centroid) in objects.items():
                if objectID not in attention_counts:
                    attention_counts[objectID] = 0
                    total_counts[objectID] = 0

                for (x, y, w, h) in faces:
                    if abs(centroid[0] - (x + w // 2)) < w // 2 and abs(centroid[1] - (y + h // 2)) < h // 2:
                        face_img = frame[y:y+h, x:x+w]
                        if classify_frame(model, face_img):
                            attention_counts[objectID] += 1
                        total_counts[objectID] += 1

            frame_number += frame_step
        else:
            frame_number += pause_frames

    cap.release()

    attention_percentages = {objectID: (attention_counts[objectID] / total_counts[objectID]) * 100
                             for objectID in attention_counts}

    return attention_percentages

# Path to the video file (needs to be adjusted)
video_path = '/content/drive/MyDrive/My Drive/sample_vid.mp4'

# Calculate attention percentages
start_time = time.time()
attention_percentages = calculate_attention_percentage(video_path)
end_time = time.time()

# Print the attention percentages
print("Attention Percentages for each user:")
for user_id, percentage in attention_percentages.items():
    print(f"User {user_id}: {percentage:.2f}%")

print(f"Total Processing Time: {end_time - start_time} seconds")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
1/1 [==============================] - 0s 32ms/step
Attention Percentages for each user:
User 0: 100.00%
User 1: 92.31%
User 2: 100.00%
User 3: 100.00%
User 4: 100.00%
User 5: 96.00%
User 6: 64.86%
User 7: 93.75%
User 8: 100.00%
User 9: 100.00%
User 10: 100.00%
User 11: 100.00%
User 12: 100.00%
User 13: 100.00%
User 14: 95.45%
User 15: 67.86%
User 16: 84.85%
User 17: 100.00%
User 18: 100.00%
User 19: 87.88%
Total Processing Time: 132.21982383728027 seconds


In [18]:
import json

# Your existing code to calculate attention percentages
attention_percentages = calculate_attention_percentage(video_path)

# Convert the attention percentages to JSON format
attention_data_json = json.dumps(attention_percentages)

print("Attention Percentages for each user:")
for user_id, percentage in attention_percentages.items():
    print(f"User {user_id}: {percentage:.2f}%")

print(f"Total Processing Time: {end_time - start_time} seconds")


1/1 [==============================] - 0s 35ms/step
Attention Percentages for each user:
User 0: 100.00%
User 1: 92.31%
User 2: 100.00%
User 3: 100.00%
User 4: 100.00%
User 5: 96.00%
User 6: 64.86%
User 7: 93.75%
User 8: 100.00%
User 9: 100.00%
User 10: 100.00%
User 11: 100.00%
User 12: 100.00%
User 13: 100.00%
User 14: 95.45%
User 15: 67.86%
User 16: 84.85%
User 17: 100.00%
User 18: 100.00%
User 19: 87.88%
Total Processing Time: 132.21982383728027 seconds


In [19]:
print(attention_percentages)

{0: 100.0, 1: 92.3076923076923, 2: 100.0, 3: 100.0, 4: 100.0, 5: 96.0, 6: 64.86486486486487, 7: 93.75, 8: 100.0, 9: 100.0, 10: 100.0, 11: 100.0, 12: 100.0, 13: 100.0, 14: 95.45454545454545, 15: 67.85714285714286, 16: 84.84848484848484, 17: 100.0, 18: 100.0, 19: 87.87878787878788}


In [23]:
from IPython.core.display import display, HTML

# Define the HTML and JavaScript for the popup
html_code = f"""
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Analytics Popup</title>
    <style>
        body {{
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            background-color: #2c2c2c;
            color: #f0f0f0;
            margin: 0;
            padding: 0;
        }}
        #openPopupBtn {{
            background-color: #444;
            color: #fff;
            border: none;
            padding: 10px 20px;
            font-size: 16px;
            cursor: pointer;
            border-radius: 5px;
            transition: background-color 0.3s ease;
            position: fixed;
            top: 20px;
            left: 20px;
        }}
        #openPopupBtn:hover {{
            background-color: #666;
        }}
        #popup {{
            display: none;
            position: fixed;
            z-index: 1;
            left: 0;
            top: 0;
            width: 100%;
            height: 100%;
            overflow: auto;
            background-color: rgba(0, 0, 0, 0.75);
            backdrop-filter: blur(5px);
        }}
        .popup-content {{
            background-color: #3a3a3a;
            margin: 10% auto;
            padding: 20px;
            border: 1px solid #888;
            width: 80%;
            max-width: 600px;
            box-shadow: 0 5px 15px rgba(0, 0, 0, 0.5);
            border-radius: 10px;
        }}
        .close-btn {{
            color: #bbb;
            float: right;
            font-size: 28px;
            font-weight: bold;
            cursor: pointer;
            transition: color 0.3s ease;
        }}
        .close-btn:hover,
        .close-btn:focus {{
            color: #fff;
            text-decoration: none;
        }}
        h2 {{
            margin-top: 0;
            border-bottom: 2px solid #555;
            padding-bottom: 10px;
        }}
        #analyticsContent {{
            margin-top: 20px;
            max-height: 400px;
            overflow-y: auto;
        }}
        .user-analytics {{
            background-color: #555;
            margin: 10px 0;
            padding: 10px;
            border-radius: 5px;
            transition: background-color 0.3s ease;
        }}
        .user-analytics:hover {{
            background-color: #777;
        }}
        ::-webkit-scrollbar {{
            width: 10px;
        }}
        ::-webkit-scrollbar-track {{
            background: #3a3a3a;
        }}
        ::-webkit-scrollbar-thumb {{
            background: #555;
        }}
        ::-webkit-scrollbar-thumb:hover {{
            background: #777;
        }}
    </style>
</head>
<body>
    <button id="openPopupBtn">Show Analytics</button>
    <div id="popup" class="popup">
        <div class="popup-content">
            <span class="close-btn">&times;</span>
            <h2>Attention Analytics</h2>
            <div id="analyticsContent"></div>
        </div>
    </div>
    <script>
        const attentionData = {attention_data_json};

        document.addEventListener("DOMContentLoaded", function() {{
            const openPopupBtn = document.getElementById("openPopupBtn");
            const popup = document.getElementById("popup");
            const closeBtn = document.querySelector(".close-btn");
            const analyticsContent = document.getElementById("analyticsContent");

            openPopupBtn.style.display = "block";

            openPopupBtn.addEventListener("click", function() {{
                displayAnalytics(attentionData);
                popup.style.display = "block";
            }});

            closeBtn.addEventListener("click", function() {{
                popup.style.display = "none";
            }});

            window.addEventListener("click", function(event) {{
                if (event.target === popup) {{
                    popup.style.display = "none";
                }}
            }});

            function displayAnalytics(data) {{
                analyticsContent.innerHTML = ""; // Clear previous content
                for (const [user, percentage] of Object.entries(data)) {{
                    const userDiv = document.createElement("div");
                    userDiv.className = "user-analytics";
                    userDiv.innerHTML = `<strong>${{user}}</strong>: ${{percentage.toFixed(2)}}% attention`;
                    analyticsContent.appendChild(userDiv);
                }}
            }}
        }});
    </script>
</body>
</html>
"""

# Create an iframe to embed the HTML and JavaScript
iframe_code = f"""
<iframe srcdoc="{html_code.replace('"', '&quot;')}" style="width: 100%; height: 500px; border: none;"></iframe>
"""

# Display the iframe with the embedded HTML and JavaScript
display(HTML(iframe_code))
